In [58]:
from coinor.grumpy.polyhedron2D import Polyhedron2D, Figure
import gurobipy as gu
import numpy as np

In [59]:
# utility function to get the model as np arrays
def get_model_arrays(mdl):
    
    # we assume that the model is Ax >= b
    for c in mdl.getConstrs():
        if c.sense == gu.GRB.LESS_EQUAL:
            c.setAttr(gu.GRB.Attr.Sense, gu.GRB.GREATER_EQUAL)
            c.setAttr(gu.GRB.Attr.RHS, -c.rhs)
    
    # get the constraints
    A = mdl.getA().toarray()
    b = np.array([c.rhs for c in mdl.getConstrs()])
    
    # get the finite lower bounds on variables
    Dlb, Dlb_0 = np.eye(mdl.numVars), np.array([v.lb for v in mdl.getVars()])
    Dlb, Dlb_0 = Dlb[Dlb_0 > -np.inf], Dlb_0[Dlb_0 > -np.inf]
    
    # get the finite upper bounds on variables
    Dub, Dub_0 = -np.eye(mdl.numVars), -np.array([v.ub for v in mdl.getVars()])
    Dub, Dub_0 = Dub[Dub_0 > -np.inf], Dub_0[Dub_0 > -np.inf]
    
    # merge the variable bounds into a single matrix with the constraints
    A = np.vstack([A, Dlb, Dub])
    b = np.hstack([b, Dlb_0, Dub_0])
    
    # get the objective function
    c = np.array([v.obj for v in mdl.getVars()])
    
    return A, b, c


# utility function to get the disjunctive constraints as np arrays
def get_disjunctive_constraint_arrays(mdl):
    
    # we assume that the model is Ax >= b
    for c in mdl.getConstrs():
        if c.sense == gu.GRB.LESS_EQUAL:
            c.setAttr(gu.GRB.Attr.Sense, gu.GRB.GREATER_EQUAL)
            c.setAttr(gu.GRB.Attr.RHS, -c.rhs)
    
    # get the disjunctive constraints from branching up
    Dlb, Dlb_0 = np.eye(mdl.numVars), np.array([v.lb for v in mdl.getVars()])
    Dlb, Dlb_0 = Dlb[Dlb_0 > 0], Dlb_0[Dlb_0 > 0]
    
    # get the disjunctive constraints from branching down
    Dub, Dub_0 = -np.eye(mdl.numVars), -np.array([v.ub for v in mdl.getVars()])
    Dub, Dub_0 = Dub[Dub_0 > -np.inf], Dub_0[Dub_0 > -np.inf]
    
    # merge the variable bounds into a single matrix with the constraints
    D = np.vstack([Dlb, Dub])
    D_0 = np.hstack([Dlb_0, Dub_0])
    
    return D, D_0

def get_tableau_primitives(mdl, original_objective=None):
    
    if original_objective is not None:
        assert len(original_objective) == mdl.numVars, \
            "original objective must be the same length as the number of variables"
    
    # create empty containers to hold basis and nonbasis information
    # basis is always m x m since matrix is always m x (n + m)
    A_b = np.zeros((mdl.numConstrs, mdl.numConstrs))
    A_n = np.zeros((mdl.numConstrs, mdl.numVars))
    c_b = np.zeros(mdl.numConstrs)  # always m
    c_n = np.zeros(mdl.numVars)  # always n since we have n + m variables and m are basic
    k_b, k_n = 0, 0  # counter for the number of basic and nonbasic variables
    
    # get the necessary model pieces as arrays
    A = mdl.getA().toarray()
    c = original_objective if original_objective is not None \
        else np.array([v.obj for v in mdl.getVars()])
    
    # populate it with columns from basic decision variables
    for i, v in enumerate(mdl.getVars()):
        if v.vBasis == 0:
            A_b[:, k_b] = A[:, i]
            c_b[k_b] = c[i]
            k_b += 1
        else:
            A_n[:, k_n] = A[:, i]
            c_n[k_n] = c[i]
            k_n += 1
            
    # populate it with columns from basic slack variables
    for i, c in enumerate(mdl.getConstrs()):
        if c.cBasis == 0:
            A_b[i, k_b] = -1
            c_b[k_b] = 0
            k_b += 1
        else:
            A_n[i, k_n] = -1
            c_n[k_n] = 0
            k_n += 1
    
    return A_b, c_b, A_n, c_n

# Define and Solve a Feasible Parent Node's LP

In [60]:
mdl = gu.Model("lp_minimize")

# Create variables
x = mdl.addVar(name="x")
y = mdl.addVar(name="y")

# Set objective function: maximize x + y + 10z
mdl.setObjective(-x - 2*y, gu.GRB.MINIMIZE)

# Add constraints
constr_0 = mdl.addConstr(x - y >= 0, "constr_0")
constr_1 = mdl.addConstr(-x - y >= -3, "constr_1")
constr_2 = mdl.addConstr(x >= 0, "constr_2")
constr_3 = mdl.addConstr(y >= 0, "constr_2")

mdl.update()

# get the model as np arrays
A0 = mdl.getA().toarray()
b0 = np.array([c.rhs for c in mdl.getConstrs()])
c0 = np.array([v.obj for v in mdl.getVars()])

# make sure this works
mdl.optimize()
x.x, y.x

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x66f85cf1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 4 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -4.500000000e+00


(1.5, 1.5)

# Find the Parent Optimal Basis Cone

In [61]:
# get the tableau primitives for the root optimal basis
A_b, c_b, A_n, c_n = get_tableau_primitives(mdl)

In [62]:
# see which decision and slack variables are in the basis (i.e. are not at their bounds - marked by 0 value)
mdl.vBasis, mdl.cBasis

([0, 0], [-1, -1, 0, 0])

In [63]:
# original basis cone
cone_0 = -(np.linalg.inv(A_b) @ A_n)
cone_0

array([[ 0.5, -0.5],
       [-0.5, -0.5],
       [ 0.5, -0.5],
       [-0.5, -0.5]])

In [64]:
c_b @ (np.linalg.inv(A_b) @ A_n) - c_n

array([-0.5, -1.5])

In [65]:
A0

array([[ 1., -1.],
       [-1., -1.],
       [ 1.,  0.],
       [ 0.,  1.]])

# Repeat for Left Child

In [66]:
# Set branching down constraint X <= 1
mdl_1 = mdl.copy()
x, y = mdl_1.getVarByName('x'), mdl_1.getVarByName('y')
constr_1_3 = mdl_1.addConstr(-x >= -1, "constr_1_3")

# find the new solution
mdl_1.optimize()

# get the model as np arrays
A1 = mdl_1.getA().toarray()
b1 = np.array([c.rhs for c in mdl_1.getConstrs()])
c1 = np.array([v.obj for v in mdl_1.getVars()])

x.x, y.x

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 2 columns and 7 nonzeros
Model fingerprint: 0x5325852e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 5 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -3.000000000e+00


(1.0, 1.0)

# Find the Left Child Optimal Basis Cone

In [67]:
# get the tableau primitives for the root optimal basis
A_b, c_b, A_n, c_n = get_tableau_primitives(mdl_1)

In [68]:
# see which decision and slack variables are in the basis (i.e. are not at their bounds - marked by 0 value)
mdl_1.vBasis, mdl_1.cBasis

([0, 0], [-1, 0, 0, 0, -1])

In [69]:
# original basis cone
cone_1 = -(np.linalg.inv(A_b) @ A_n)
cone_1

array([[-0., -1.],
       [-1., -1.],
       [ 1.,  2.],
       [-0., -1.],
       [-1., -1.]])

In [70]:
c_b @ (np.linalg.inv(A_b) @ A_n) - c_n

array([-2., -3.])

In [75]:
np.linalg.inv(A_b) @ b1

array([1., 1., 1., 1., 1.])

In [72]:
np.linalg.inv(A_b)

array([[ 0.,  0.,  0.,  0., -1.],
       [-1., -0., -0., -0., -1.],
       [ 1., -1., -0., -0.,  2.],
       [ 0.,  0., -1.,  0., -1.],
       [-1., -0., -0., -1., -1.]])

In [76]:
A_n

array([[-1.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0., -1.]])

# Repeat for Right Child

In [77]:
# Set branching down constraint x >= 2
mdl_2 = mdl.copy()
x, y = mdl_2.getVarByName('x'), mdl_2.getVarByName('y')
constr_2_3 = mdl_2.addConstr(x >= 2, "constr_2_3")

# find the new solution
mdl_2.optimize()

# get the model as np arrays
A2 = mdl_2.getA().toarray()
b2 = np.array([c.rhs for c in mdl_2.getConstrs()])
c2 = np.array([v.obj for v in mdl_2.getVars()])

x.x, y.x

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 2 columns and 7 nonzeros
Model fingerprint: 0x33b6857a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 3e+00]
Presolve removed 5 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -4.000000000e+00


(2.0, 1.0)

# Find the Right Child Optimal Basis Cone

In [78]:
# get the tableau primitives for the root optimal basis
A_b, c_b, A_n, c_n = get_tableau_primitives(mdl_2)

In [79]:
# see which decision and slack variables are in the basis (i.e. are not at their bounds - marked by 0 value)
mdl_2.vBasis, mdl_2.cBasis

([0, 0], [0, -1, 0, 0, -1])

In [80]:
# original basis cone
cone_2 = -(np.linalg.inv(A_b) @ A_n)
cone_2

array([[-0.,  1.],
       [-1., -1.],
       [ 1.,  2.],
       [-0.,  1.],
       [-1., -1.]])

In [81]:
A_n

array([[ 0.,  0.],
       [-1.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0., -1.]])

In [82]:
c_b @ (np.linalg.inv(A_b) @ A_n) - c_n

array([-2., -1.])

In [83]:
np.linalg.inv(A_b) @ b2

array([2., 1., 1., 2., 1.])

In [84]:
np.linalg.inv(A_b)

array([[ 0.,  0.,  0.,  0.,  1.],
       [-0., -1., -0., -0., -1.],
       [-1.,  1.,  0.,  0.,  2.],
       [-0., -0., -1., -0.,  1.],
       [-0., -1., -0., -1., -1.]])

In [85]:
b2

array([ 0., -3.,  0.,  0.,  2.])